<a href="https://colab.research.google.com/github/zohaibkhanzohaibi/NIS-CCP/blob/main/Frequency_Attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Frequency Attack

In [ ]:
# Python implementation of the frequency attack for the composite cipher described
# - Encrypt/Decrypt functions implement the "for each key letter: Caesar shift by k_r, then Vigenere with full key" process.
# - Kasiski and IOC to estimate period m.
# - Column chi-squared frequency analysis to recover effective Vigenère shifts K_eff.
# - Modular solving (with gcd handling) to recover original key letters k_t from K_eff.
# - Candidate generation, decryption and scoring, and example on the user's Key = 'AB' sample.

# This code runs end-to-end on a ciphertext string and prints candidate plaintexts and keys.
# No external internet required.

In [ ]:
from collections import Counter, defaultdict
import math
import itertools
import re

# ----------------------------- Utilities ----------------------------------
ALPH = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
A2I = {c: i for i, c in enumerate(ALPH)}  # A->0
I2A = {i: c for i, c in enumerate(ALPH)}  # 0->A

ENGLISH_FREQ = {
    'A': 0.08167, 'B': 0.01492, 'C': 0.02782, 'D': 0.04253, 'E': 0.12702, 'F': 0.02228,
    'G': 0.02015, 'H': 0.06094, 'I': 0.06966, 'J': 0.00153, 'K': 0.00772, 'L': 0.04025,
    'M': 0.02406, 'N': 0.06749, 'O': 0.07507, 'P': 0.01929, 'Q': 0.00095, 'R': 0.05987,
    'S': 0.06327, 'T': 0.09056, 'U': 0.02758, 'V': 0.00978, 'W': 0.02360, 'X': 0.00150,
    'Y': 0.01974, 'Z': 0.00074
}

COMMON_WORDS = ["THE","AND","TO","OF","IN","IS","YOU","THAT","IT","HE","WAS","FOR","ON","ARE","AS","WITH","HIS","THEY","I"]

def normalize(text):
    return "".join(ch for ch in text.upper() if ch.isalpha())

def caesar_shift_text(text, shift):
    # shift: integer (0..25) where positive shifts forward (A->B)
    return "".join(I2A[(A2I[ch] + shift) % 26] for ch in text)

def vigenere_encrypt(text, key):
    # key: list of ints 0..25
    out = []
    for i, ch in enumerate(text):
        k = key[i % len(key)]
        out.append(I2A[(A2I[ch] + k) % 26])
    return "".join(out)

def vigenere_decrypt(text, key):
    out = []
    for i, ch in enumerate(text):
        k = key[i % len(key)]
        out.append(I2A[(A2I[ch] - k) % 26])
    return "".join(out)

# composite encryption: for each key letter in order, do Caesar shift by k_r, then Vigenere with full key
def composite_encrypt(plaintext, key_letters):  # key_letters: list of ints 0..25
    text = normalize(plaintext)
    key = key_letters
    for kr in key:
        # Caesar by kr
        text = caesar_shift_text(text, kr)
        # Vigenere by full key
        text = vigenere_encrypt(text, key)
    return text

# composite decryption reversing the order: for each key letter reversed: Vigenere decrypt with full key, then Caesar by -k_r
def composite_decrypt(ciphertext, key_letters):
    text = normalize(ciphertext)
    key = key_letters
    for kr in reversed(key):
        text = vigenere_decrypt(text, key)
        text = caesar_shift_text(text, -kr)
    return text

In [ ]:
# --------------------------- Kasiski & IOC --------------------------------
def find_repeats(ciphertext, min_len=3, max_len=5):
    s = normalize(ciphertext)
    repeats = defaultdict(list)
    for L in range(min_len, max_len+1):
        seen = {}
        for i in range(len(s)-L+1):
            sub = s[i:i+L]
            if sub in seen:
                repeats[sub].append(i - seen[sub])
            else:
                seen[sub] = i
    # Flatten to distances for frequency of factors
    distances = []
    for sub, dists in repeats.items():
        for d in dists:
            if d > 0:
                distances.append(d)
    return distances

def factor_counts(n):
    # return non-trivial factors of n (<= 30 for our period search)
    facs = set()
    for f in range(2, 51):
        if n % f == 0:
            facs.add(f)
    return facs

def kasiski_candidates(ciphertext):
    distances = find_repeats(ciphertext)
    fact_count = Counter()
    for d in distances:
        for f in factor_counts(d):
            fact_count[f] += 1
    if not fact_count:
        return []
    # return most common factor candidates
    return [cand for cand, _ in fact_count.most_common(10)]

def index_of_coincidence(text):
    s = normalize(text)
    n = len(s)
    if n <= 1:
        return 0.0
    freq = Counter(s)
    ic = sum(v*(v-1) for v in freq.values()) / (n*(n-1))
    return ic

def ioc_for_period(ciphertext, period):
    s = normalize(ciphertext)
    cols = [''.join(s[i::period]) for i in range(period)]
    ics = [index_of_coincidence(c) for c in cols if c]
    return sum(ics)/len(ics) if ics else 0.0

def ioc_candidates(ciphertext, max_period=20):
    scores = []
    for p in range(1, max_period+1):
        scores.append((p, ioc_for_period(ciphertext, p)))
    # return top candidates by IOC
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:8]

In [ ]:
# --------------------------- Frequency analysis ---------------------------
def chi_squared_stat(observed_counts, expected_freq, total):
    # expected_freq: dict A->freq (sums to 1)
    chi = 0.0
    for ch in ALPH:
        expected = expected_freq.get(ch, 0) * total
        observed = observed_counts.get(ch, 0)
        if expected > 0:
            chi += (observed - expected)**2 / expected
    return chi

def best_shift_for_column(column):
    # try all 26 shifts to see which shift produces letter distribution closest to English
    n = len(column)
    if n == 0:
        return 0, 0.0  # no data
    counts = Counter(column)
    best = None
    for s in range(26):
        # shifting column backward by s simulates candidate key shift s (i.e., decrypt by s)
        shifted_counts = Counter()
        for ch, cnt in counts.items():
            shifted_ch = I2A[(A2I[ch] - s) % 26]
            shifted_counts[shifted_ch] += cnt
        chi = chi_squared_stat(shifted_counts, ENGLISH_FREQ, n)
        if best is None or chi < best[0]:
            best = (chi, s)
    return best[1], best[0]  # return shift s (0..25) and chi value

In [ ]:
# --------------------------- Modular solving ------------------------------
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def inv_mod(a, m):
    g, x, _ = egcd(a % m, m)
    if g != 1:
        return None
    else:
        return x % m

def solve_for_S_total(sum_K, m):
    # Solve 2m * S_total ≡ sum_K (mod 26)
    coef = (2*m) % 26
    g = math.gcd(coef, 26)
    solutions = []
    if sum_K % g != 0:
        return []
    # reduce equation by g
    coef_r = coef // g
    mod_r = 26 // g
    rhs_r = (sum_K // g) % mod_r
    inv = inv_mod(coef_r, mod_r)
    if inv is None:
        # handle as enumeration (should be small)
        for cand in range(26):
            if (coef * cand - sum_K) % 26 == 0:
                solutions.append(cand % 26)
    else:
        s0 = (rhs_r * inv) % mod_r
        # lift to all g solutions
        for k in range(g):
            solutions.append((s0 + k*mod_r) % 26)
    return sorted(set(solutions))

def solve_for_k_t(K_eff_t, S_total, m):
    # Solve m * k_t ≡ (K_eff_t - S_total) (mod 26)
    rhs = (K_eff_t - S_total) % 26
    g = math.gcd(m, 26)
    sols = []
    if rhs % g != 0:
        return []
    m_r = m // g
    mod_r = 26 // g
    rhs_r = (rhs // g) % mod_r
    inv = inv_mod(m_r, mod_r)
    if inv is None:
        # enumerate
        for cand in range(26):
            if (m * cand - K_eff_t + S_total) % 26 == 0:
                sols.append(cand % 26)
    else:
        k0 = (rhs_r * inv) % mod_r
        for t in range(g):
            sols.append((k0 + t*mod_r) % 26)
    return sorted(set(sols))

In [ ]:
# --------------------------- Attack pipeline ------------------------------
def recover_effective_key(ciphertext, m):
    s = normalize(ciphertext)
    K_eff = []
    chi_vals = []
    for t in range(m):
        col = s[t::m]
        shift, chi = best_shift_for_column(col)
        # shift is the amount that decrypts column => that is the effective key letter for that column
        K_eff.append(shift % 26)
        chi_vals.append(chi)
    return K_eff, chi_vals

def score_plaintext(pt):
    # simple scoring: count common words + letter frequency chi-sq penalty (lower better)
    s = pt.upper()
    score_words = sum(s.count(" " + w + " ") + s.startswith(w+" ") + s.endswith(" "+w) or ((" " + w + " ") in (" " + s + " ")) for w in COMMON_WORDS)
    # For a simpler robust metric, use count of common words occurrences
    word_hits = sum(s.count(w) for w in COMMON_WORDS)
    # letter frequency chi-squared
    counts = Counter(s.replace(" ", ""))
    chi = chi_squared_stat(counts, ENGLISH_FREQ, max(1, len(s.replace(" ", ""))))
    # combine:
    # prefer more word hits and lower chi
    return word_hits * 50 - chi  # tunable

def attack(ciphertext, max_period=12, top_periods=5, beam_per_column=1):
    ctext = normalize(ciphertext)
    # 1) Kasiski and IOC suggestions
    kas = kasiski_candidates(ctext)
    ioc_cands = ioc_candidates(ctext, max_period=max_period)
    ioc_sorted = [p for p,_ in ioc_cands]
    period_candidates = []
    # merge heuristics
    for cand in (kas + ioc_sorted):
        if cand not in period_candidates and 1 <= cand <= max_period:
            period_candidates.append(cand)
    # fill with 1..max_period if empty
    if not period_candidates:
        period_candidates = list(range(1, max_period+1))
    period_candidates = period_candidates[:top_periods]
    results = []
    for m in period_candidates:
        # 2) recover effective key K_eff by column frequency
        K_eff, chis = recover_effective_key(ctext, m)
        sumK = sum(K_eff) % 26
        S_total_candidates = solve_for_S_total(sumK, m)
        if not S_total_candidates:
            # fallback: try brute-forcing S_total 0..25
            S_total_candidates = list(range(26))
        # For each S_total, solve for k_t possibilities and enumerate combinations (cartesian product)
        all_key_candidates = []
        for S_total in S_total_candidates:
            per_col_candidates = []
            for t in range(m):
                sols = solve_for_k_t(K_eff[t], S_total, m)
                if not sols:
                    sols = list(range(26))  # fallback
                per_col_candidates.append(sols)
            # combine; but limit total combinations to reasonable number
            combos = list(itertools.product(*per_col_candidates))
            # reduce combos if too many
            if len(combos) > 2000:
                combos = combos[:2000]
            for combo in combos:
                all_key_candidates.append((S_total, combo))
        # Now try decrypt each candidate key and score
        for S_total, key_tuple in all_key_candidates:
            key_list = list(key_tuple)
            pt = composite_decrypt(ctext, key_list)
            sc = score_plaintext(pt)
            results.append({
                'period': m,
                'K_eff': K_eff,
                'S_total': S_total,
                'key': key_list,
                'plaintext': pt,
                'score': sc
            })
    # sort results by score descending
    results.sort(key=lambda r: r['score'], reverse=True)
    # deduplicate by plaintext
    seen = set()
    output = []
    for r in results:
        if r['plaintext'] not in seen:
            seen.add(r['plaintext'])
            output.append(r)
        if len(output) >= 10:
            break
    return output, period_candidates, ioc_cands

In [ ]:
#    # Ram limit exceeds here and memory gets crashed.
# --------------------------- Demonstration --------------------------------
# if __name__ == "__main__":
#     plaintext = "While Google Docs does not have a built-in function to directly generate 'random text' in the sense of a paragraph of placeholder text (like 'Lorem ipsum'), you can achieve this in Google Sheets and then copy it into Docs, or use a Google Apps Script in Docs for more complex random text generation."
#     # User mapping A->1, B->2 corresponds to 0-based ints A->0,B->1 but the user's derivation used 1..26.
#     # Our implementation uses 0..25 for key letters. The user's key "AB" will be represented as [0,1].
#     key_str = "AB"
#     key_letters = [A2I[ch] for ch in key_str]  # A->0, B->1
#     print("Plaintext:", plaintext)
#     print("Key letters (0-based):", key_letters)
#     ct = composite_encrypt(plaintext, key_letters)
#     print("Ciphertext produced by composite_encrypt:", ct)
#     print("\n--- Running attack pipeline on generated ciphertext ---\n")
#     results, period_candidates, ioc_cands = attack(ct, max_period=8, top_periods=6)
#     print("Period candidates tried:", period_candidates)
#     print("IOC top scores (period, IOC):", ioc_cands)
#     print("\nTop candidates found:")
#     for idx, r in enumerate(results, 1):
#         k_readable = ''.join(I2A[k] for k in r['key'])
#         print(f"\nCandidate #{idx}: score={r['score']:.2f}, period={r['period']}, recovered_key={k_readable} ({r['key']})")
#         print("Plaintext:", r['plaintext'])
#     # If nothing plausible found, show the top 3 tries by period + K_eff for debugging
#     if not results:
#         for m in period_candidates[:3]:
#             K_eff, chis = recover_effective_key(ct, m)
#             print(f"\nDebug period {m}: K_eff={K_eff}, chis={chis}")

# Solution for Efficient memory usage using heap


In [ ]:
# --- Memory-safe, beam-search based attack pipeline ---
import heapq
import time

def top_shifts_for_column(column, top_k=4):
    """Return top_k candidate shifts for this column sorted by chi (lowest first).
       Each item is (shift, chi)."""
    n = len(column)
    if n == 0:
        return [(0, 1e9)]
    counts = Counter(column)
    cand_list = []
    for s in range(26):
        shifted_counts = Counter()
        for ch, cnt in counts.items():
            shifted_ch = I2A[(A2I[ch] - s) % 26]
            shifted_counts[shifted_ch] += cnt
        chi = chi_squared_stat(shifted_counts, ENGLISH_FREQ, n)
        cand_list.append((s, chi))
    cand_list.sort(key=lambda x: x[1])
    return cand_list[:top_k]

def beam_search_keys_from_Keff(K_eff, m, beam_width=200, top_shifts_per_col=3,
                               max_combos_per_period=20000, time_limit_sec=10):
    """
    Build candidate original keys (k_0..k_{m-1}) using beam search to avoid explosion.
    We use top_shifts_per_col candidates per column (from chi analysis) and then solve for
    S_total candidates; for each S_total we perform a constrained beam across columns.
    Yields (S_total, key_tuple) candidates one by one (streaming).
    """
    start_time = time.time()
    sumK = sum(K_eff) % 26
    S_total_candidates = solve_for_S_total(sumK, m)
    if not S_total_candidates:
        S_total_candidates = list(range(26))
    combos_yielded = 0

    for S_total in S_total_candidates:
        # For each column compute candidate k_t solutions (but limit to top by heuristic)
        per_col_solutions = []
        for t in range(m):
            sols = solve_for_k_t(K_eff[t], S_total, m)
            if not sols:
                sols = list(range(26))
            # rank sols by closeness to expected small shifts (heuristic: prefer small values)
            # but better: rank by plausibility - here we just limit to the first top_shifts_per_col
            # to keep things small. If sols > top_shifts_per_col, pick most "likely" ones randomly or heuristically.
            if len(sols) > top_shifts_per_col:
                sols = sols[:top_shifts_per_col]
            per_col_solutions.append(sols)

        # Beam search: combine columns incrementally
        beam = [()]  # each entry is tuple of partial key pieces
        total_considered = 0
        for t in range(m):
            next_beam = []
            for partial in beam:
                for candidate in per_col_solutions[t]:
                    new_partial = partial + (candidate,)
                    next_beam.append(new_partial)
                    total_considered += 1
                    if total_considered >= max_combos_per_period:
                        break
                if total_considered >= max_combos_per_period:
                    break
            # prune beam by heuristic: decrypt partial as far as possible? Here we keep smallest beams by lexicographic
            # better heuristic: score decrypt of partial applied over positions so far, but that's more compute.
            # Simple pruning:
            if len(next_beam) > beam_width:
                next_beam = next_beam[:beam_width]
            beam = next_beam
            # time check
            if time.time() - start_time > time_limit_sec:
                break

        # yield complete keys from beam
        for full_key in beam:
            if len(full_key) == m:
                combos_yielded += 1
                yield (S_total, full_key)
                if combos_yielded >= max_combos_per_period:
                    return
            if time.time() - start_time > time_limit_sec:
                return

def attack_memory_safe(ciphertext, max_period=12, top_periods=5,
                       top_shifts_per_col=3, beam_width_per_period=300,
                       max_results=10, max_combos_per_period=20000, time_limit_sec=12):
    """
    Memory-safe attack:
      - limit candidate periods
      - per period: recover K_eff by chi, then beam-search key candidates
      - keep top max_results using a heap
    """
    ctext = normalize(ciphertext)
    kas = kasiski_candidates(ctext)
    ioc_cands = ioc_candidates(ctext, max_period=max_period)
    ioc_sorted = [p for p,_ in ioc_cands]
    period_candidates = []
    for cand in (kas + ioc_sorted):
        if cand not in period_candidates and 1 <= cand <= max_period:
            period_candidates.append(cand)
    if not period_candidates:
        period_candidates = list(range(1, max_period+1))
    period_candidates = period_candidates[:top_periods]

    # Min-heap for top results (store negative score for max-heap effect)
    heap = []
    total_checked = 0
    start = time.time()

    for m in period_candidates:
        if time.time() - start > time_limit_sec:
            break
        K_eff, chis = recover_effective_key(ctext, m)
        # For each column we can also preselect top candidate shifts using chi-scores:
        # (we will use that info inside beam generator via solve_for_k_t which may be smaller)
        # use beam search generator to produce candidate keys
        gen = beam_search_keys_from_Keff(K_eff, m, beam_width=beam_width_per_period,
                                        top_shifts_per_col=top_shifts_per_col,
                                        max_combos_per_period=max_combos_per_period,
                                        time_limit_sec=max(1, time_limit_sec//len(period_candidates)))
        for S_total, key_tuple in gen:
            key_list = list(key_tuple)
            pt = composite_decrypt(ctext, key_list)
            sc = score_plaintext(pt)
            total_checked += 1
            # push onto heap
            if len(heap) < max_results:
                heapq.heappush(heap, (sc, m, K_eff, S_total, key_list, pt))
            else:
                # if this candidate better than worst in heap, replace
                if sc > heap[0][0]:
                    heapq.heapreplace(heap, (sc, m, K_eff, S_total, key_list, pt))
            if time.time() - start > time_limit_sec:
                break
        # free memory hints
        del K_eff, chis
        if time.time() - start > time_limit_sec:
            break

    # produce final sorted list (best first)
    best = sorted(heap, key=lambda x: x[0], reverse=True)
    results = []
    for sc, m, K_eff, S_total, key_list, pt in best:
        results.append({
            'score': sc,
            'period': m,
            'K_eff': K_eff,
            'S_total': S_total,
            'key': key_list,
            'plaintext': pt
        })
    return results, period_candidates, ioc_cands, total_checked


In [ ]:
# --------------------------- Demonstration (memory-efficient) -------------------------------
if __name__ == "__main__":
    plaintext = (
        "Giving help that's not asked for is what makes a true hero!"
        "If you keep looking down on everyone, then you won’t know your own weakness."
        "The words I can say to you are: look properly at what you want to be"
        "Someone who can't sacrifice anything, can't ever change anything"
        "My Soldiers, Rage! My Soldiers, Scream! My Soldiers, Fight!"
    )

    # key_str in 0..25 mapping (A->0, B->1, ...)
    key_str = "AB"
    key_letters = [A2I[ch] for ch in key_str]  # A->0, B->1

    print("Plaintext (preview):", plaintext[:120] + "...")
    print("Key letters (0-based):", key_letters)
    ct = composite_encrypt(plaintext, key_letters)
    print("Ciphertext produced by composite_encrypt (preview):", ct[:120] + "...")
    print("\n--- Running memory-safe attack pipeline on generated ciphertext ---\n")

    # run the memory-safe attack (tunable params)
    results, period_candidates, ioc_info, checked = attack_memory_safe(
        ciphertext=ct,
        max_period=12,
        top_periods=6,
        top_shifts_per_col=3,    # 2-4 is a good starting point
        beam_width_per_period=200,
        max_results=8,
        max_combos_per_period=5000,
        time_limit_sec=12
    )

    print("Period candidates tried:", period_candidates)
    print("IOC top scores (period, IOC):", ioc_info)
    print("Total candidate keys checked (approx):", checked)
    print("\nTop candidates found:")

    if results:
        for idx, r in enumerate(results, 1):
            k_readable = ''.join(I2A[k] for k in r['key'])
            print(f"\nCandidate #{idx}: score={r['score']:.2f}, period={r['period']}, recovered_key={k_readable} ({r['key']})")
            print("Plaintext (preview):", (r['plaintext'][:200] + "...") if len(r['plaintext']) > 200 else r['plaintext'])
    else:
        print("No high-scoring candidates found. Showing K_eff debug for first few periods.")
        for m in period_candidates[:3]:
            K_eff, chis = recover_effective_key(ct, m)
            print(f"\nDebug period {m}: K_eff={K_eff}, chis={chis}")


Plaintext (preview): Giving help that's not asked for is what makes a true hero!If you keep looking down on everyone, then you won’t know you...
Key letters (0-based): [0, 1]
Ciphertext produced by composite_encrypt (preview): HLWLOJIHMSUKBWTQPWBVLHEIPUJVXKBWNDLHTDUUVHIHSRJIZRVNFHQOPRLLOJERXQPQFYFUZROHUKFQZRVZPQUNORXBPXSRXQXHBNOHTVUKFZPUEVJFBQTD...

--- Running memory-safe attack pipeline on generated ciphertext ---

Period candidates tried: [2, 3, 6, 7, 5, 10]
IOC top scores (period, IOC): [(2, 0.06016240157480315), (8, 0.05972782258064516), (10, 0.05961538461538461), (4, 0.05803571428571429), (6, 0.0576173369707119), (12, 0.05577200577200577), (5, 0.05058220211161387), (9, 0.05055849500293944)]
Total candidate keys checked (approx): 707

Top candidates found:

Candidate #1: score=1954.72, period=2, recovered_key=AB ([0, 1])
Plaintext (preview): GIVINGHELPTHATSNOTASKEDFORISWHATMAKESATRUEHEROIFYOUKEEPLOOKINGDOWNONEVERYONETHENYOUWONTKNOWYOUROWNWEAKNESSTHEWORDSICANSAYTOYOUARELOOKPROPERLY